In [4]:
import pandas as pd
import numpy as np
import glob
import os

In [8]:
def get_combined_df(repo_dir):
    all_files = glob.glob(os.path.join(repo_dir, '*.parquet'))
    all_files.sort()
    all_dataframes = [pd.read_parquet(file) for file in all_files]
    combined_df = pd.concat(all_dataframes, ignore_index=True)
    combined_df['commit_date'] = (combined_df['commit_date'].astype('int64') / 1e9).astype('int64')
    # replace NaN with empty string
    # combined_df.fillna('', inplace=True)
    return combined_df

In [97]:
repo_dir = '../data/final_fbr/'
fbdf = get_combined_df(repo_dir)

In [98]:
fbdf.shape

(69835, 14)

In [61]:
fbdf.head()

,owner,repo_name,commit_date,commit_id,commit_message,file_path,cur_file_content,previous_commit_id,previous_file_path,previous_file_content,diff,status,is_merge_request,file_extension
0,facebook,react,1696026278,a6ed60a8eb0626e5f84d0bdbb62c0b61219150d3,[Fizz] Don't double replay elements when it's ...,packages/react-dom/src/__tests__/ReactDOMFizzS...,"/** * Copyright (c) Meta Platforms, Inc. and ...",c7ba8c098889b6dc47fa9c807bbba3975a658584,<NA>,"/** * Copyright (c) Meta Platforms, Inc. and ...","@@ -6988,4 +6988,100 @@ describe('ReactDOMFizz...",modified,False,js
1,facebook,react,1696026278,a6ed60a8eb0626e5f84d0bdbb62c0b61219150d3,[Fizz] Don't double replay elements when it's ...,packages/react-server/src/ReactFizzServer.js,"/** * Copyright (c) Meta Platforms, Inc. and ...",c7ba8c098889b6dc47fa9c807bbba3975a658584,<NA>,"/** * Copyright (c) Meta Platforms, Inc. and ...","@@ -487,7 +487,7 @@ export function resumeRequ...",modified,False,js
2,facebook,react,1696026245,c7ba8c098889b6dc47fa9c807bbba3975a658584,"Enforce that the ""react-server"" build of ""reac...",.eslintrc.js,"'use strict'; const { es5Paths, esNextPat...",d900fadbf9017063fecb2641b7e99303b82a6f17,<NA>,"'use strict'; const { es5Paths, esNextPat...","@@ -517,6 +517,7 @@ module.exports = { __...",modified,False,js
3,facebook,react,1696026245,c7ba8c098889b6dc47fa9c807bbba3975a658584,"Enforce that the ""react-server"" build of ""reac...",packages/react-client/src/__tests__/ReactFligh...,"/** * Copyright (c) Meta Platforms, Inc. and ...",d900fadbf9017063fecb2641b7e99303b82a6f17,<NA>,"/** * Copyright (c) Meta Platforms, Inc. and ...","@@ -14,6 +14,7 @@ let act; let use; let star...",modified,False,js
4,facebook,react,1696026245,c7ba8c098889b6dc47fa9c807bbba3975a658584,"Enforce that the ""react-server"" build of ""reac...",packages/react-server-dom-turbopack/src/__test...,"/** * Copyright (c) Meta Platforms, Inc. and ...",d900fadbf9017063fecb2641b7e99303b82a6f17,<NA>,"/** * Copyright (c) Meta Platforms, Inc. and ...","@@ -62,7 +62,7 @@ describe('ReactFlightDOM', (...",modified,False,js


In [21]:
# show counts by status
fbdf.groupby('status').count()

/var/folders/00/mwsht4n55dl_7kj9p52w9j940000gn/T/ipykernel_38974/1818841972.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  fbdf.groupby('status').count()


,owner,repo_name,commit_date,commit_id,commit_message,file_path,cur_file_content,previous_commit_id,previous_file_path,previous_file_content,diff,is_merge_request,file_extension
status,,,,,,,,,,,,,
added,5600,5600,5600,5600,5600,5600,5600,0,0,0,0,5600,5600
deleted,2669,2669,2669,2669,2669,2669,0,2669,0,2669,0,2669,2669
modified,57451,57451,57451,57451,57451,57451,57451,57451,0,57451,57450,57451,57451
renamed,4140,4140,4140,4140,4140,4140,4140,4140,4140,4140,4140,4140,4140


In [99]:
# show counts by status
fbdf.groupby('status').count()

/var/folders/00/mwsht4n55dl_7kj9p52w9j940000gn/T/ipykernel_38974/1818841972.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  fbdf.groupby('status').count()


,owner,repo_name,commit_date,commit_id,commit_message,file_path,cur_file_content,previous_commit_id,previous_file_path,previous_file_content,diff,is_merge_request,file_extension
status,,,,,,,,,,,,,
added,5588,5588,5588,5588,5588,5588,5588,0,0,0,0,5588,5588
deleted,2656,2656,2656,2656,2656,2656,0,2656,0,2656,0,2656,2656
modified,57451,57451,57451,57451,57451,57451,57451,57451,0,57451,57450,57451,57451
renamed,4140,4140,4140,4140,4140,4140,4140,4140,4140,4140,4140,4140,4140


In [63]:
# see sample of status == 'renamed'
fbdf[fbdf['status'] == 'renamed'].head()

,owner,repo_name,commit_date,commit_id,commit_message,file_path,cur_file_content,previous_commit_id,previous_file_path,previous_file_content,diff,status,is_merge_request,file_extension
21,facebook,react,1696026245,c7ba8c098889b6dc47fa9c807bbba3975a658584,"Enforce that the ""react-server"" build of ""reac...",packages/react/src/ReactSharedInternalsClient.js,"/** * Copyright (c) Meta Platforms, Inc. and ...",d900fadbf9017063fecb2641b7e99303b82a6f17,packages/react/src/ReactSharedInternals.js,"/** * Copyright (c) Meta Platforms, Inc. and ...","@@ -0,0 +1,33 @@ +/** + * Copyright (c) Meta P...",renamed,False,js
126,facebook,react,1695833611,701ac2e57290f913502c6012c15e815f412f84b7,[Flight][Float] Preinitialize module imports d...,packages/react-server-dom-esm/src/ReactFlightC...,"/** * Copyright (c) Meta Platforms, Inc. and ...",49eba01930e9e1f331b34967fca65d5a0ba62846,packages/react-server-dom-esm/src/ReactFlightC...,"/** * Copyright (c) Meta Platforms, Inc. and ...","@@ -0,0 +1,120 @@ +/** + * Copyright (c) Meta ...",renamed,False,js
131,facebook,react,1695833611,701ac2e57290f913502c6012c15e815f412f84b7,[Flight][Float] Preinitialize module imports d...,packages/react-server-dom-webpack/src/ReactFli...,"/** * Copyright (c) Meta Platforms, Inc. and ...",49eba01930e9e1f331b34967fca65d5a0ba62846,packages/react-server-dom-webpack/src/ReactFli...,"/** * Copyright (c) Meta Platforms, Inc. and ...","@@ -0,0 +1,162 @@ +/** + * Copyright (c) Meta ...",renamed,False,js
132,facebook,react,1695833611,701ac2e57290f913502c6012c15e815f412f84b7,[Flight][Float] Preinitialize module imports d...,packages/react-server-dom-webpack/src/ReactFli...,"/** * Copyright (c) Meta Platforms, Inc. and ...",49eba01930e9e1f331b34967fca65d5a0ba62846,packages/react-server-dom-webpack/src/ReactFli...,"/** * Copyright (c) Meta Platforms, Inc. and ...","@@ -0,0 +1,234 @@ +/** + * Copyright (c) Meta ...",renamed,False,js
462,facebook,react,1693307366,8fbd3079425eaff85ec1b52b0eefecdd44ba7289,refactor: refactored devtools browser extensio...,packages/react-devtools-extensions/src/content...,"/** * Copyright (c) Meta Platforms, Inc. and ...",f47956239f6892a1e805af1fc3f1b2fb0f66beca,packages/react-devtools-extensions/src/backend...,"/** * Copyright (c) Meta Platforms, Inc. and ...","@@ -0,0 +1,179 @@ +/** + * Copyright (c) Meta ...",renamed,False,js


In [51]:
print(fbdf.query("commit_id == 'c7ba8c098889b6dc47fa9c807bbba3975a658584' & file_path == 'packages/react-server-dom-turbopack/src/__tests__/ReactFlightDOMNode-test.js'")['diff'].values[0])

@@ -42,7 +42,7 @@ describe('ReactFlightDOMNode', () => {
     turbopackModuleLoading = TurbopackMock.moduleLoading;
 
     jest.resetModules();
-    jest.unmock('react');
+    __unmockReact();
     jest.unmock('react-server-dom-turbopack/server');
     jest.mock('react-server-dom-turbopack/client', () =>
       require('react-server-dom-turbopack/client.node'),


In [58]:
fbdf.query("commit_id == '5e0dfdac54e509a97483af1f78af09451c03bfbb'").shape

(29, 14)

In [88]:
fbdf.query("commit_id == '069d7d7e72e73c9b8c0d6b22e77073e227c72a60'")

,owner,repo_name,commit_date,commit_id,commit_message,file_path,previous_commit_id,previous_file_content,cur_file_content,diff,status,is_merge_request,file_extension


In [89]:
import git

In [91]:
fbRepo = git.Repo('../repos/facebook_react')

In [96]:
# get commit 069d7d7e72e73c9b8c0d6b22e77073e227c72a60

commit = fbRepo.commit('069d7d7e72e73c9b8c0d6b22e77073e227c72a60')

ValueError: SHA b'069d7d7e72e73c9b8c0d6b22e77073e227c72a60' could not be resolved, git returned: b'069d7d7e72e73c9b8c0d6b22e77073e227c72a60 missing'